# Stage 0: Preparation


In [1]:
import tiktoken
import dotenv
import os
dotenv.load_dotenv()
# OR
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

OPENAI_MODEL = "gpt-3.5-turbo"

encoder = tiktoken.get_encoding("cl100k_base")

# Stage 1: LLM initialization

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=OPENAI_MODEL,
    callbacks=[],  # Will be used in agents part
    streaming=True,  # Streaming mode - response will be streamed token-by-token
    temperature=0.5,  # Temperature of LLM response generation
    max_tokens=1024  # Output tokens limit
)

In [3]:
result = llm.invoke("Hello, world!").content
print(result)
print(f'IN tokens: {len(encoder.encode("Hello, world!"))}, OUT tokens: {len(encoder.encode(result))}')


Hello! How can I assist you today?
IN tokens: 4, OUT tokens: 9


# Stage 2: Prompt Engineering

In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
   template="Give responses to the following question: {question}. Be brief and give easy to understand responses. Your response should be in {language}",
   input_variables=["question", "language"]
)
prompt_formatted_str: str = prompt.format(
   question="Who is Santa Claus?",
   language="Ukrainian"
)
print(prompt_formatted_str)

Give responses to the following question: Who is Santa Claus?. Be brief and give easy to understand responses. Your response should be in Ukrainian


In [5]:
prediction = llm.invoke(prompt_formatted_str).content
print(prediction)

Санта Клаус - це вигаданий чоловік, який розносить подарунки дітям у святкову ніч.


# Stage 3: Chains

In [6]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.chains import LLMMathChain

prompt = PromptTemplate(
   template="Generate a simple math task for adding {entity}.",
   input_variables=["entity"]
)
math_chain = LLMMathChain.from_llm(llm=llm)
chain = prompt | llm | math_chain


In [7]:
chain.invoke("apples", config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "apples"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "input": "apples"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Generate a simple math task for adding apples."
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.06s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "If you have 5 apples and you buy 3 more apples, how many apples do you have in total?",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIM

{'question': AIMessage(content='If you have 5 apples and you buy 3 more apples, how many apples do you have in total?', response_metadata={'finish_reason': 'stop'}, id='run-54587afa-6a3b-4ba3-a5be-84cb9afac320-0'),
 'answer': 'Answer: 8'}

# Stage 4: Agents and Tools

In [8]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, Tool, load_tools

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

wikipedia_tool = Tool(
    name="Wikipedia",
    func= WikipediaAPIWrapper().run,
	description="A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions."
)

tools_set = [
    wikipedia_tool,
    load_tools(["llm-math"], llm=llm)[0]
]

agent = create_tool_calling_agent(
    tools=tools_set,
    llm=llm,
    prompt=prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools_set, verbose=True)


In [9]:
result = agent_executor.invoke({"input" :"Add radius of Earth to radius of Moon and multiply by 2"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `(6371 + 1737) * 2`


Answer: 16216
Invoking: `Wikipedia` with `radius of Earth`


Page: Earth radius
Summary: Earth radius (denoted as R🜨 or 
  
    
      
        
          R
          
            E
          
        
      
    
    {\displaystyle R_{E}}
  
) is the distance from the center of Earth to a point on or near its surface. Approximating the figure of Earth by an Earth spheroid, the radius ranges from a maximum of nearly 6,378 km (3,963 mi) (equatorial radius, denoted a) to a minimum of nearly 6,357 km (3,950 mi) (polar radius, denoted b). 
A nominal Earth radius is sometimes used as a unit of measurement in astronomy and geophysics, which is recommended by the International Astronomical Union to be the equatorial value.
A globally-average value is usually considered to be 6,371 kilometres (3,959 mi) with a 0.3% variability (±10 km) for the following reasons.
The International Union of Geodesy and Geo

In [10]:
print(result['output'])

The sum of the radius of Earth and the radius of the Moon multiplied by 2 is 16,216 kilometers.

The radius of Earth is the distance from the center of Earth to a point on or near its surface. The nominal Earth radius is usually considered to be 6,371 kilometers with a variability of about ±110 kilometers.


# Stage 5: Conversation Memory, Chat

In [11]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_with_memory = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=False
)
print(conversation_with_memory.predict(input="Hi!"))
print(conversation_with_memory.predict(input="What was my previous message?"))

Hello! How are you today?
Your previous message was "Hi!"


In [13]:
llm.invoke("What was my previous question?").content

'Your previous question was "What is the capital of France?"'

# Stage 6: Vector Stores and Similarity Search

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

raw_documents = TextLoader('./story.txt').load()
text_splitter = CharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0
)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [15]:
query = 'Who said "No, I dare not take it."?'
docs = db.similarity_search(query)
print(docs[0].page_content)

“No,” answered Snow-White; “no, I dare not take it.”

“What! are you afraid of it?” cried the old woman. “There, see—I will cut the apple in halves; do you eat the red cheeks, and I will eat the core.” (The apple was so artfully made that the red cheeks alone were poisoned.) Snow-White very much wished for the beautiful apple, and when she saw the woman eating the core she could no longer resist, but, stretching out her hand, took the poisoned part. Scarcely had she placed a piece in her mouth when she fell down dead upon the ground. Then the Queen, looking at her with glittering eyes, and laughing bitterly, exclaimed, “White as snow, red as blood, black as ebony! This time the Dwarfs cannot reawaken you.”

When she reached home and consulted her mirror—

“Mirror, mirror on the wall,
Who is the fairest of us all?”

it answered:

“The Queen is fairest of the day.”

Then her envious heart was at rest, as peacefully as an envious heart can rest.


# Stage 7: Chat Bot

In [16]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")

chat_history = ChatMessageHistory(messages=[], session_id="test-session")

agent = create_react_agent(
    llm=llm, 
    tools=tools_set, 
    prompt=prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools_set, verbose=True, handle_parsing_errors=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [17]:
agent_with_chat_history.invoke(
    {"input": "Hello"},
    config={"configurable": {"session_id": "<foo>"}},
)['output']



> Entering new AgentExecutor chain...
Hello! How can I assist you today?Invalid Format: Missing 'Action:' after 'Thought:Do I need to use a tool? No
Final Answer: No, I just wanted to say hello! How are you today?

> Finished chain.


'No, I just wanted to say hello! How are you today?'

In [18]:
agent_with_chat_history.invoke(
    {"input": "How are you?"},
    config={"configurable": {"session_id": "<foo>"}},
)['output']



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: I'm just a machine learning model, so I don't have feelings, but thank you for asking! How can I assist you today?

> Finished chain.


"I'm just a machine learning model, so I don't have feelings, but thank you for asking! How can I assist you today?"

In [19]:
agent_with_chat_history.invoke(
    {"input": "How many people live in Antarctica?"},
    config={"configurable": {"session_id": "<foo>"}},
)['output']



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Antarctica populationPage: Antarctica
Summary: Antarctica ( ) is Earth's southernmost and least-populated continent. Situated almost entirely south of the Antarctic Circle and surrounded by the Southern Ocean (also known as the Antarctic Ocean), it contains the geographic South Pole. Antarctica is the fifth-largest continent, being about 40% larger than Europe, and has an area of 14,200,000 km2 (5,500,000 sq mi). Most of Antarctica is covered by the Antarctic ice sheet, with an average thickness of 1.9 km (1.2 mi).
Antarctica is, on average, the coldest, driest, and windiest of the continents, and it has the highest average elevation. It is mainly a polar desert, with annual precipitation of over 200 mm (8 in) along the coast and far less inland. About 70% of the world's freshwater reserves are frozen in Antarctica, which, if melted, would raise global sea levels by almost 60 m

'Approximately 4,000 people live in Antarctica during the summer months, and this number drops to around 1,000 in the winter.'

In [20]:
agent_with_chat_history.invoke(
    {"input": "What was my first question?"},
    config={"configurable": {"session_id": "<foo>"}},
)['output']



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: "How many people live in Antarctica?"Page: Antarctica
Summary: Antarctica ( ) is Earth's southernmost and least-populated continent. Situated almost entirely south of the Antarctic Circle and surrounded by the Southern Ocean (also known as the Antarctic Ocean), it contains the geographic South Pole. Antarctica is the fifth-largest continent, being about 40% larger than Europe, and has an area of 14,200,000 km2 (5,500,000 sq mi). Most of Antarctica is covered by the Antarctic ice sheet, with an average thickness of 1.9 km (1.2 mi).
Antarctica is, on average, the coldest, driest, and windiest of the continents, and it has the highest average elevation. It is mainly a polar desert, with annual precipitation of over 200 mm (8 in) along the coast and far less inland. About 70% of the world's freshwater reserves are frozen in Antarctica, which, if melted, would raise global sea level

'Your first question was "How many people live in Antarctica?" and the answer is that during the summer months, about 5,000 people reside at research stations, a figure that drops to around 1,000 in the winter.'